In [1]:
import os 
import pandas as pd 
import csv 
import cv2 
import numpy as np 

In [2]:
path = "/content/drive/MyDrive/Colab Notebooks/Datasets/dataset/"
labels_path = path + "driving_log.csv"
images_data_path = path + "IMG/"


In [3]:
df = pd.read_csv(labels_path)
df.columns = ["image_center","image_left","image_right","steering_angle","throttle","break","speed"]
df.head()

,image_center,image_left,image_right,steering_angle,throttle,break,speed
0,E:\dataset\IMG\center_2022_06_17_10_57_26_046.jpg,E:\dataset\IMG\left_2022_06_17_10_57_26_046.jpg,E:\dataset\IMG\right_2022_06_17_10_57_26_046.jpg,0.0,0.000000,0,0.000013
1,E:\dataset\IMG\center_2022_06_17_10_57_26_133.jpg,E:\dataset\IMG\left_2022_06_17_10_57_26_133.jpg,E:\dataset\IMG\right_2022_06_17_10_57_26_133.jpg,0.0,0.000000,0,0.000019
2,E:\dataset\IMG\center_2022_06_17_10_57_26_216.jpg,E:\dataset\IMG\left_2022_06_17_10_57_26_216.jpg,E:\dataset\IMG\right_2022_06_17_10_57_26_216.jpg,0.0,0.000000,0,0.000002
3,E:\dataset\IMG\center_2022_06_17_10_57_26_303.jpg,E:\dataset\IMG\left_2022_06_17_10_57_26_303.jpg,E:\dataset\IMG\right_2022_06_17_10_57_26_303.jpg,0.0,0.000000,0,0.000013
4,E:\dataset\IMG\center_2022_06_17_10_57_26_466.jpg,E:\dataset\IMG\left_2022_06_17_10_57_26_466.jpg,E:\dataset\IMG\right_2022_06_17_10_57_26_466.jpg,0.0,0.483081,0,0.711340


In [4]:
lines = [] 
with open(labels_path) as csvfile:
  reader = csv.reader(csvfile)
  
  for line in reader:
    lines.append(line)
images = [] 
measurements = [] 
print("Started Reading Data from Dataset")
for line in lines:
  image_name = line[0].split("\\")[-1]
  image_path = images_data_path + image_name 
  image = cv2.imread(image_path)
  measurement = float(line[3])
  images.append(image)
  measurements.append(measurement)
print("Done Reading Data From Datasets")

Started Reading Data from Dataset
Done Reading Data From Datasets


In [7]:
shape = images[0].shape
print(shape)

(160, 320, 3)


In [8]:
augmented_images = []
augmented_measurements = [] 
for image,measurement in zip(images,measurements):
   augmented_images.append(image)
   augmented_measurements.append(measurement)
   augmented_images.append(cv2.flip(image,1))
   augmented_measurements.append(measurement*-1.0)

In [11]:
x_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

In [19]:
from keras.models import Sequential 
from keras.layers import Dense,Flatten,Lambda,Cropping2D,Dropout
from keras.layers.convolutional import Convolution2D

model  = Sequential()
model.add(Lambda(lambda x:x/255.0 - 0.5,input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(2,3,input_shape = (24,5,5),strides=(2,2),activation="relu"))
model.add(Dropout(0.2))
model.add(Convolution2D(2,3,input_shape = (36,5,5),strides=(2,2),activation="relu"))
model.add(Dropout(0.2))
model.add(Convolution2D(2,3,input_shape = (48,5,5),strides=(2,2),activation="relu"))
model.add(Dropout(0.2))
# model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Convolution2D(2,3,input_shape = (64,3,3),activation="relu"))
model.add(Dropout(0.2))
# model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Convolution2D(2,3,input_shape = (64,3,3),activation="relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.5))
model.add(Dense(50))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Dense(1))
model.compile(loss="mse",optimizer="adam")
model.summary()




Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_7 (Lambda)           (None, 160, 320, 3)       0         
                                                                 
 cropping2d_7 (Cropping2D)   (None, 65, 320, 3)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 32, 159, 2)        56        
                                                                 
 dropout_5 (Dropout)         (None, 32, 159, 2)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 15, 79, 2)         38        
                                                                 
 dropout_6 (Dropout)         (None, 15, 79, 2)         0         
                                                                 
 conv2d_8 (Conv2D)           (None, 7, 39, 2)         

In [21]:
model.fit(x_train,y_train,epochs=3,validation_split=0.2,shuffle=True)  

Epoch 1/3
83/83 [==============================] - 8s 91ms/step - loss: 0.0547 - val_loss: 0.0083
Epoch 2/3
83/83 [==============================] - 8s 92ms/step - loss: 0.0548 - val_loss: 0.0085
Epoch 3/3
83/83 [==============================] - 8s 92ms/step - loss: 0.0548 - val_loss: 0.0082


In [22]:
model.save("model.h5")